In [15]:
import pandas as pd
import numpy as np
import ast
import swifter
from tqdm.notebook import tqdm

In [16]:
fossology_copyrights_df = pd.read_csv('fossologyAPI_CSVs/fossology-master.csv')
scan_code_copyrights_df = pd.read_csv('scan_code_csvs/fossology-master.csv')

In [17]:
print('API Length: ', len(fossology_copyrights_df))
print('Scan code Length: ', len(scan_code_copyrights_df))

API Length:  43743
Scan code Length:  4399


In [18]:
fossology_copyrights_df = fossology_copyrights_df.loc[:, ['file_path', 'copyright']]
scan_code_copyrights_df = scan_code_copyrights_df.loc[:, ['file_path', 'copyrights']]
scan_code_copyrights_df = scan_code_copyrights_df.rename(columns={'copyrights': 'scan_code_copyrights'})

In [19]:
fossology_copyrights_df = fossology_copyrights_df.sort_values('file_path')
scan_code_copyrights_df = scan_code_copyrights_df.sort_values('file_path')

In [20]:
fossology_copyrights_df = fossology_copyrights_df.reset_index()
fossology_copyrights_df = fossology_copyrights_df.loc[:, ['file_path', 'copyright']]
scan_code_copyrights_df = scan_code_copyrights_df.reset_index()
scan_code_copyrights_df = scan_code_copyrights_df.loc[:, ['file_path', 'scan_code_copyrights']]

In [21]:
# Fix scan_code_copyrights being string even though they should be lists
def convert_scan_code_copyrights_to_list(row):
    s = row['scan_code_copyrights']
    l = ast.literal_eval(s)
    row['scan_code_copyrights'] = l
    return row

scancode_df = scan_code_copyrights_df.swifter.progress_bar(True).apply(convert_scan_code_copyrights_to_list, axis=1)

Pandas Apply:   0%|          | 0/4399 [00:00<?, ?it/s]

In [22]:
fossology_copyrights_df.head()

,file_path,copyright
0,fossology-master/.dockerignore,copyright/agent_tests/Unit/test_copyright src/...
1,fossology-master/.dockerignore,copyright/VERSION-copyright src/spdx2/agent_te...
2,fossology-master/.dockerignore,copyright_list src/cli/fo_folder src/cli/fo_no...
3,fossology-master/.dockerignore,copyright/VERSION-keyword
4,fossology-master/.dockerignore,copyright/VERSION-ecc


In [23]:
scan_code_copyrights_df.head()

,file_path,scan_code_copyrights
0,fossology-master/.dockerignore,[Copyright (c) Fossology contributors]
1,fossology-master/.editorconfig,[Copyright (c) Fossology contributors]
2,fossology-master/.github/ISSUE_TEMPLATE.md,[Copyright (c) Fossology contributors]
3,fossology-master/.github/ISSUE_TEMPLATE/bug_re...,[Copyright (c) Fossology contributors]
4,fossology-master/.github/ISSUE_TEMPLATE/featur...,[Copyright (c) Fossology contributors]


In [24]:
print('API Length: ', len(fossology_copyrights_df))
print('Scancode Length: ', len(scan_code_copyrights_df))

API Length:  43743
Scancode Length:  4399


In [25]:
merged_df = fossology_copyrights_df.join(scan_code_copyrights_df.set_index('file_path'), on='file_path', how='left')

In [26]:
merged_df['falsePositive'] = 1

In [27]:
def check_false_positive(row):
    try:
        if row['scan_code_copyrights'] is np.nan:
            return row
        for scan_code_copyright in row['scan_code_copyrights']:
            lower_scan_code = scan_code_copyright.lower()
            lower_foss_copyright = row['copyright'].lower()
            if lower_scan_code in lower_foss_copyright:
                row['scan_code_copyrights'] = scan_code_copyright
                row['falsePositive'] = 0
                break
            elif lower_scan_code.replace('copyright (c)', '©') in lower_foss_copyright:
                row['scan_code_copyrights'] = scan_code_copyright
                row['falsePositive'] = 0
                break
            elif lower_scan_code.replace('copyright (c)', '(c)') in lower_foss_copyright:
                row['scan_code_copyrights'] = scan_code_copyright
                row['falsePositive'] = 0
                break
            elif lower_scan_code.replace('copyright', '(c)') in lower_foss_copyright:
                row['scan_code_copyrights'] = scan_code_copyright
                row['falsePositive'] = 0
                break
            elif lower_scan_code.replace('copyright', '©') in lower_foss_copyright:
                row['scan_code_copyrights'] = scan_code_copyright
                row['falsePositive'] = 0
                break
            elif lower_scan_code.replace('(c)', '©') in lower_foss_copyright:
                row['scan_code_copyrights'] = scan_code_copyright
                row['falsePositive'] = 0
                break
            elif lower_scan_code.replace('(c)', 'copyright ©') in lower_foss_copyright:
                row['scan_code_copyrights'] = scan_code_copyright
                row['falsePositive'] = 0
                break
            elif lower_scan_code.replace('(c)', 'copyright') in lower_foss_copyright:
                row['scan_code_copyrights'] = scan_code_copyright
                row['falsePositive'] = 0
                break
            elif lower_scan_code.replace('(c)', 'copyright (c)') in lower_foss_copyright:
                row['scan_code_copyrights'] = scan_code_copyright
                row['falsePositive'] = 0
                break
            else:
                row['scan_code_copyrights'] = '-'
    except Exception as E:
        print(E, row['scan_code_copyrights'])
    return row

In [28]:
merged_df = merged_df.swifter.progress_bar(enable=True, desc=None).apply(check_false_positive, axis=1)

Pandas Apply:   0%|          | 0/43743 [00:00<?, ?it/s]

In [ ]:
def remove_lists(row):
    if type(row['scan_code_copyrights']) == list:
        row['scan_code_copyrights'] = '-'
    if row['scan_code_copyrights'] == '':
        row['scan_code_copyrights'] = '-'
    return row

In [ ]:
#merged_df = merged_df.swifter.progress_bar(enable=True, desc=None).apply(remove_lists, axis=1)

Pandas Apply:   0%|          | 0/43743 [00:00<?, ?it/s]

In [16]:
merged_df.to_csv('merged_csvs/fossology-master.csv', index=False)

In [19]:
merged_df.iloc[0]['scan_code_copyrights']

"['Copyright (c) Fossology contributors']"